In [1]:
import pandas as pd
import numpy as np
import catboost as cgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics
import time
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [2]:
dftrain = pd.read_csv('Adult/adult.0.train')
dftest = pd.read_csv('Adult/adult.0.test')


In [3]:
colname = ['ordinal', 'nominal', 'ordinal1', 'nominal2', 'ordinal3', 'nominal4', 'nominal5', 'nominal6', 'nominal7', 'nominal8', 'ordinal9', 'ordinal10', 'ordinal11', 'nominal12', 'output']

In [4]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24420 entries, 0 to 24419
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   65              24420 non-null  int64 
 1   Private         24420 non-null  object
 2   170939          24420 non-null  int64 
 3   Bachelors       24420 non-null  object
 4   13              24420 non-null  int64 
 5   Divorced        24420 non-null  object
 6   Prof-specialty  24420 non-null  object
 7   Not-in-family   24420 non-null  object
 8   White           24420 non-null  object
 9   Male            24420 non-null  object
 10  6723            24420 non-null  int64 
 11  0               24420 non-null  int64 
 12  40              24420 non-null  int64 
 13  United-States   24420 non-null  object
 14  <=50K           24420 non-null  object
dtypes: int64(6), object(9)
memory usage: 2.8+ MB


In [6]:
dftrain.columns = colname
dftest.columns = colname

In [7]:
dftrain['nominal12'].replace({'Holand-Netherlands':999999999}, inplace = True)

In [8]:
dftrain.replace({'?':'NAN'}, inplace=True)
dftest.replace({'?':'NAN'}, inplace=True)

In [9]:
nominal = ['nominal', 'nominal2', 'nominal4', 'nominal5', 'nominal6', 'nominal7', 'nominal8', 'nominal12']

In [10]:
dftrain.shape, dftest.shape

((24420, 15), (24420, 15))

In [11]:
X_train = dftrain.drop(['output'], axis = 1)
X_test = dftest.drop(['output'], axis = 1)
y_train = dftrain['output']
y_test = dftest['output']

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((24420, 14), (24420,), (24420, 14), (24420,))

In [14]:
def hypertune(model, param):
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=6, random_state=36)
        grid_search = GridSearchCV(model, param, cv=cv, scoring = ('accuracy'), return_train_score = True, n_jobs = -1, 
                             verbose = 2)
        
        start = time.time()
        grid_search.fit(X_train, y_train)
        stop = time.time()
        sec = stop-start
        minu = sec/60
        hrs = minu/60
        print("\n")
        print('Total time required for execution:', hrs)
        print("\n")
        print('best parameters are:', grid_search.best_params_)
        print('CV accuracy:', grid_search.best_score_)
        print("\n")
        print("The cross validation results are:", grid_search.cv_results_)

In [18]:
model1 = CatBoostClassifier(iterations=100,learning_rate=0.1, depth=8,l2_leaf_reg=1,random_strength=1,
                           border_count=254,bagging_temperature = 1, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
#params1 = {'learning_rate':[0.1,0.2,0.3], 'iterations': range(100, 550, 50)}
#params1 = {'learning_rate':[0.001,0.01,0.1], 'iterations':range(150,250,25)}
#params1 = {'iterations':range(180,225,10)}
params1 = {'iterations':range(190,211,5)}
#hypertune(model1,params1)

Fitting 30 folds for each of 5 candidates, totalling 150 fits
0:	learn: 0.8395168	total: 26.4ms	remaining: 5.26s
1:	learn: 0.8468468	total: 46ms	remaining: 4.56s
2:	learn: 0.8472154	total: 70.7ms	remaining: 4.64s
3:	learn: 0.8511876	total: 99.1ms	remaining: 4.86s
4:	learn: 0.8518837	total: 122ms	remaining: 4.78s
5:	learn: 0.8548321	total: 146ms	remaining: 4.71s
6:	learn: 0.8558559	total: 168ms	remaining: 4.63s
7:	learn: 0.8564292	total: 190ms	remaining: 4.55s
8:	learn: 0.8576986	total: 213ms	remaining: 4.51s
9:	learn: 0.8586814	total: 237ms	remaining: 4.5s
10:	learn: 0.8583129	total: 261ms	remaining: 4.48s
11:	learn: 0.8597461	total: 284ms	remaining: 4.46s
12:	learn: 0.8591728	total: 310ms	remaining: 4.46s
13:	learn: 0.8602375	total: 336ms	remaining: 4.46s
14:	learn: 0.8602375	total: 363ms	remaining: 4.47s
15:	learn: 0.8607289	total: 386ms	remaining: 4.44s
16:	learn: 0.8605651	total: 411ms	remaining: 4.42s
17:	learn: 0.8604423	total: 436ms	remaining: 4.4s
18:	learn: 0.8615479	total: 46

164:	learn: 0.8959459	total: 4.38s	remaining: 930ms
165:	learn: 0.8960688	total: 4.41s	remaining: 904ms
166:	learn: 0.8963554	total: 4.44s	remaining: 878ms
167:	learn: 0.8966011	total: 4.47s	remaining: 851ms
168:	learn: 0.8967240	total: 4.5s	remaining: 825ms
169:	learn: 0.8968878	total: 4.53s	remaining: 799ms
170:	learn: 0.8976658	total: 4.56s	remaining: 773ms
171:	learn: 0.8975020	total: 4.58s	remaining: 746ms
172:	learn: 0.8979934	total: 4.61s	remaining: 720ms
173:	learn: 0.8981572	total: 4.64s	remaining: 693ms
174:	learn: 0.8986486	total: 4.67s	remaining: 667ms
175:	learn: 0.8986896	total: 4.7s	remaining: 640ms
176:	learn: 0.8986896	total: 4.73s	remaining: 614ms
177:	learn: 0.8988124	total: 4.75s	remaining: 587ms
178:	learn: 0.8990991	total: 4.78s	remaining: 561ms
179:	learn: 0.8995086	total: 4.81s	remaining: 534ms
180:	learn: 0.8993857	total: 4.83s	remaining: 508ms
181:	learn: 0.8996314	total: 4.86s	remaining: 481ms
182:	learn: 0.8998362	total: 4.89s	remaining: 455ms
183:	learn: 0.

In [21]:
model2 = CatBoostClassifier(iterations=200,learning_rate=0.1, depth=8,l2_leaf_reg=1,random_strength=1,
                           border_count=254,bagging_temperature = 1, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
params2 = {'depth':[4,5,6,7]}
#hypertune(model2,params2)

Fitting 30 folds for each of 4 candidates, totalling 120 fits
0:	learn: 0.8384111	total: 15.9ms	remaining: 3.16s
1:	learn: 0.8434889	total: 28.8ms	remaining: 2.85s
2:	learn: 0.8445946	total: 42.5ms	remaining: 2.79s
3:	learn: 0.8481572	total: 54.4ms	remaining: 2.67s
4:	learn: 0.8507371	total: 65.8ms	remaining: 2.56s
5:	learn: 0.8504095	total: 75.3ms	remaining: 2.43s
6:	learn: 0.8506552	total: 84.8ms	remaining: 2.34s
7:	learn: 0.8524570	total: 95.4ms	remaining: 2.29s
8:	learn: 0.8525389	total: 107ms	remaining: 2.27s
9:	learn: 0.8540131	total: 119ms	remaining: 2.25s
10:	learn: 0.8536036	total: 133ms	remaining: 2.29s
11:	learn: 0.8548321	total: 146ms	remaining: 2.29s
12:	learn: 0.8555283	total: 160ms	remaining: 2.3s
13:	learn: 0.8556102	total: 174ms	remaining: 2.31s
14:	learn: 0.8556921	total: 187ms	remaining: 2.31s
15:	learn: 0.8556102	total: 200ms	remaining: 2.3s
16:	learn: 0.8561835	total: 213ms	remaining: 2.29s
17:	learn: 0.8559787	total: 227ms	remaining: 2.3s
18:	learn: 0.8560606	tota

170:	learn: 0.8778460	total: 2.47s	remaining: 419ms
171:	learn: 0.8778460	total: 2.48s	remaining: 404ms
172:	learn: 0.8780098	total: 2.5s	remaining: 390ms
173:	learn: 0.8780098	total: 2.51s	remaining: 376ms
174:	learn: 0.8778870	total: 2.53s	remaining: 361ms
175:	learn: 0.8780098	total: 2.54s	remaining: 346ms
176:	learn: 0.8780508	total: 2.56s	remaining: 332ms
177:	learn: 0.8780098	total: 2.57s	remaining: 318ms
178:	learn: 0.8779689	total: 2.58s	remaining: 303ms
179:	learn: 0.8781736	total: 2.6s	remaining: 289ms
180:	learn: 0.8782965	total: 2.61s	remaining: 274ms
181:	learn: 0.8784193	total: 2.63s	remaining: 260ms
182:	learn: 0.8782555	total: 2.64s	remaining: 246ms
183:	learn: 0.8783374	total: 2.66s	remaining: 231ms
184:	learn: 0.8784603	total: 2.67s	remaining: 217ms
185:	learn: 0.8784603	total: 2.69s	remaining: 202ms
186:	learn: 0.8784193	total: 2.7s	remaining: 188ms
187:	learn: 0.8785012	total: 2.72s	remaining: 173ms
188:	learn: 0.8787469	total: 2.73s	remaining: 159ms
189:	learn: 0.8

In [24]:
model3 = CatBoostClassifier(iterations=200,learning_rate=0.1, depth=5,l2_leaf_reg=1,random_strength=1,
                           border_count=254,bagging_temperature = 1, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
params3 = {'l2_leaf_reg':[0.4,0.5,0.6,0.7,0.8]}
#hypertune(model3,params3)

Fitting 30 folds for each of 5 candidates, totalling 150 fits
0:	learn: 0.8384111	total: 16.9ms	remaining: 3.37s
1:	learn: 0.8434889	total: 32.3ms	remaining: 3.2s
2:	learn: 0.8445946	total: 45.3ms	remaining: 2.98s
3:	learn: 0.8481572	total: 56.9ms	remaining: 2.79s
4:	learn: 0.8507371	total: 68ms	remaining: 2.65s
5:	learn: 0.8504095	total: 78.5ms	remaining: 2.54s
6:	learn: 0.8514333	total: 90.5ms	remaining: 2.49s
7:	learn: 0.8514742	total: 104ms	remaining: 2.49s
8:	learn: 0.8530303	total: 116ms	remaining: 2.46s
9:	learn: 0.8533989	total: 129ms	remaining: 2.45s
10:	learn: 0.8536036	total: 142ms	remaining: 2.44s
11:	learn: 0.8542588	total: 155ms	remaining: 2.42s
12:	learn: 0.8549550	total: 166ms	remaining: 2.39s
13:	learn: 0.8547912	total: 177ms	remaining: 2.35s
14:	learn: 0.8547912	total: 190ms	remaining: 2.34s
15:	learn: 0.8546274	total: 203ms	remaining: 2.33s
16:	learn: 0.8548731	total: 217ms	remaining: 2.33s
17:	learn: 0.8551188	total: 230ms	remaining: 2.33s
18:	learn: 0.8556511	total

168:	learn: 0.8782965	total: 2.49s	remaining: 457ms
169:	learn: 0.8786650	total: 2.51s	remaining: 442ms
170:	learn: 0.8787469	total: 2.52s	remaining: 428ms
171:	learn: 0.8787879	total: 2.54s	remaining: 413ms
172:	learn: 0.8787879	total: 2.55s	remaining: 398ms
173:	learn: 0.8790745	total: 2.56s	remaining: 383ms
174:	learn: 0.8791155	total: 2.58s	remaining: 369ms
175:	learn: 0.8791155	total: 2.6s	remaining: 354ms
176:	learn: 0.8789926	total: 2.61s	remaining: 339ms
177:	learn: 0.8790745	total: 2.63s	remaining: 324ms
178:	learn: 0.8792793	total: 2.64s	remaining: 310ms
179:	learn: 0.8793612	total: 2.66s	remaining: 295ms
180:	learn: 0.8793612	total: 2.67s	remaining: 280ms
181:	learn: 0.8793612	total: 2.69s	remaining: 266ms
182:	learn: 0.8794431	total: 2.7s	remaining: 251ms
183:	learn: 0.8794840	total: 2.72s	remaining: 236ms
184:	learn: 0.8794431	total: 2.73s	remaining: 222ms
185:	learn: 0.8795659	total: 2.75s	remaining: 207ms
186:	learn: 0.8795250	total: 2.76s	remaining: 192ms
187:	learn: 0.

In [27]:
model4 = CatBoostClassifier(iterations=200,learning_rate=0.1, depth=5,l2_leaf_reg=0.8,random_strength=1,
                           border_count=254,bagging_temperature = 1, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
#params4 = {'random_strength':[0,1,2,3,4]}
params4 = {'random_strength':[0.8,0.9,1,1.1,1.2]}
#hypertune(model4,params4)

Fitting 30 folds for each of 5 candidates, totalling 150 fits
0:	learn: 0.8384111	total: 33.9ms	remaining: 6.75s
1:	learn: 0.8434889	total: 70.5ms	remaining: 6.98s
2:	learn: 0.8445946	total: 98.9ms	remaining: 6.49s
3:	learn: 0.8481572	total: 122ms	remaining: 5.97s
4:	learn: 0.8507371	total: 144ms	remaining: 5.6s
5:	learn: 0.8504095	total: 165ms	remaining: 5.34s
6:	learn: 0.8514333	total: 183ms	remaining: 5.03s
7:	learn: 0.8514742	total: 211ms	remaining: 5.07s
8:	learn: 0.8530303	total: 237ms	remaining: 5.03s
9:	learn: 0.8533989	total: 261ms	remaining: 4.95s
10:	learn: 0.8536036	total: 282ms	remaining: 4.84s
11:	learn: 0.8542588	total: 308ms	remaining: 4.83s
12:	learn: 0.8549550	total: 331ms	remaining: 4.76s
13:	learn: 0.8547912	total: 350ms	remaining: 4.66s
14:	learn: 0.8547912	total: 367ms	remaining: 4.53s
15:	learn: 0.8546274	total: 386ms	remaining: 4.44s
16:	learn: 0.8548731	total: 406ms	remaining: 4.37s
17:	learn: 0.8551188	total: 427ms	remaining: 4.32s
18:	learn: 0.8556511	total: 

161:	learn: 0.8781736	total: 3.37s	remaining: 791ms
162:	learn: 0.8778870	total: 3.39s	remaining: 769ms
163:	learn: 0.8780098	total: 3.42s	remaining: 751ms
164:	learn: 0.8781327	total: 3.44s	remaining: 731ms
165:	learn: 0.8782555	total: 3.47s	remaining: 711ms
166:	learn: 0.8782965	total: 3.49s	remaining: 689ms
167:	learn: 0.8783784	total: 3.5s	remaining: 668ms
168:	learn: 0.8782965	total: 3.53s	remaining: 647ms
169:	learn: 0.8786650	total: 3.55s	remaining: 626ms
170:	learn: 0.8787469	total: 3.58s	remaining: 607ms
171:	learn: 0.8787879	total: 3.6s	remaining: 586ms
172:	learn: 0.8787879	total: 3.61s	remaining: 564ms
173:	learn: 0.8790745	total: 3.63s	remaining: 542ms
174:	learn: 0.8791155	total: 3.65s	remaining: 521ms
175:	learn: 0.8791155	total: 3.66s	remaining: 500ms
176:	learn: 0.8789926	total: 3.68s	remaining: 478ms
177:	learn: 0.8790745	total: 3.7s	remaining: 457ms
178:	learn: 0.8792793	total: 3.72s	remaining: 436ms
179:	learn: 0.8793612	total: 3.73s	remaining: 415ms
180:	learn: 0.8

In [29]:
model4 = CatBoostClassifier(iterations=200,learning_rate=0.1, depth=5,l2_leaf_reg=0.8,random_strength=1,
                           border_count=254,bagging_temperature = 1, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
#params4 = {'border_count':[128,254]}
params4 = {'border_count':[251,252,253,254,255]}
hypertune(model4,params4)

Fitting 30 folds for each of 5 candidates, totalling 150 fits
0:	learn: 0.8384111	total: 17.9ms	remaining: 3.55s
1:	learn: 0.8434889	total: 39ms	remaining: 3.86s
2:	learn: 0.8445946	total: 54.1ms	remaining: 3.55s
3:	learn: 0.8481572	total: 68.9ms	remaining: 3.38s
4:	learn: 0.8507371	total: 83.4ms	remaining: 3.25s
5:	learn: 0.8504095	total: 94.2ms	remaining: 3.05s
6:	learn: 0.8514333	total: 105ms	remaining: 2.9s
7:	learn: 0.8514742	total: 122ms	remaining: 2.94s
8:	learn: 0.8530303	total: 137ms	remaining: 2.9s
9:	learn: 0.8533989	total: 154ms	remaining: 2.93s
10:	learn: 0.8536036	total: 169ms	remaining: 2.9s
11:	learn: 0.8542588	total: 184ms	remaining: 2.89s
12:	learn: 0.8549550	total: 201ms	remaining: 2.89s
13:	learn: 0.8547912	total: 216ms	remaining: 2.87s
14:	learn: 0.8547912	total: 235ms	remaining: 2.9s
15:	learn: 0.8546274	total: 250ms	remaining: 2.88s
16:	learn: 0.8548731	total: 268ms	remaining: 2.88s
17:	learn: 0.8551188	total: 287ms	remaining: 2.9s
18:	learn: 0.8556511	total: 301

169:	learn: 0.8786650	total: 3.14s	remaining: 554ms
170:	learn: 0.8787469	total: 3.17s	remaining: 537ms
171:	learn: 0.8787879	total: 3.18s	remaining: 518ms
172:	learn: 0.8787879	total: 3.2s	remaining: 500ms
173:	learn: 0.8790745	total: 3.22s	remaining: 481ms
174:	learn: 0.8791155	total: 3.24s	remaining: 462ms
175:	learn: 0.8791155	total: 3.26s	remaining: 444ms
176:	learn: 0.8789926	total: 3.27s	remaining: 425ms
177:	learn: 0.8790745	total: 3.29s	remaining: 407ms
178:	learn: 0.8792793	total: 3.31s	remaining: 388ms
179:	learn: 0.8793612	total: 3.33s	remaining: 370ms
180:	learn: 0.8793612	total: 3.34s	remaining: 351ms
181:	learn: 0.8793612	total: 3.36s	remaining: 333ms
182:	learn: 0.8794431	total: 3.38s	remaining: 314ms
183:	learn: 0.8794840	total: 3.4s	remaining: 295ms
184:	learn: 0.8794431	total: 3.42s	remaining: 277ms
185:	learn: 0.8795659	total: 3.43s	remaining: 258ms
186:	learn: 0.8795250	total: 3.45s	remaining: 240ms
187:	learn: 0.8794840	total: 3.47s	remaining: 221ms
188:	learn: 0.

In [32]:
model5 = CatBoostClassifier(iterations=200,learning_rate=0.1, depth=5,l2_leaf_reg=0.8,random_strength=1,
                           border_count=254,bagging_temperature = 1, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
params5 = {'bagging_temperature':[0,0.5,0.8,0.9,1]}
hypertune(model5,params5)

Fitting 30 folds for each of 5 candidates, totalling 150 fits
0:	learn: 0.8384111	total: 15.5ms	remaining: 3.08s
1:	learn: 0.8434889	total: 30.2ms	remaining: 2.99s
2:	learn: 0.8445946	total: 44.6ms	remaining: 2.93s
3:	learn: 0.8481572	total: 55.7ms	remaining: 2.73s
4:	learn: 0.8507371	total: 67.1ms	remaining: 2.62s
5:	learn: 0.8504095	total: 77.2ms	remaining: 2.5s
6:	learn: 0.8514333	total: 89.1ms	remaining: 2.46s
7:	learn: 0.8514742	total: 103ms	remaining: 2.47s
8:	learn: 0.8530303	total: 117ms	remaining: 2.48s
9:	learn: 0.8533989	total: 129ms	remaining: 2.45s
10:	learn: 0.8536036	total: 143ms	remaining: 2.45s
11:	learn: 0.8542588	total: 156ms	remaining: 2.44s
12:	learn: 0.8549550	total: 169ms	remaining: 2.42s
13:	learn: 0.8547912	total: 182ms	remaining: 2.42s
14:	learn: 0.8547912	total: 194ms	remaining: 2.4s
15:	learn: 0.8546274	total: 209ms	remaining: 2.4s
16:	learn: 0.8548731	total: 227ms	remaining: 2.44s
17:	learn: 0.8551188	total: 242ms	remaining: 2.44s
18:	learn: 0.8556511	total

161:	learn: 0.8781736	total: 2.76s	remaining: 647ms
162:	learn: 0.8778870	total: 2.77s	remaining: 628ms
163:	learn: 0.8780098	total: 2.78s	remaining: 611ms
164:	learn: 0.8781327	total: 2.8s	remaining: 594ms
165:	learn: 0.8782555	total: 2.82s	remaining: 577ms
166:	learn: 0.8782965	total: 2.83s	remaining: 559ms
167:	learn: 0.8783784	total: 2.85s	remaining: 542ms
168:	learn: 0.8782965	total: 2.86s	remaining: 524ms
169:	learn: 0.8786650	total: 2.87s	remaining: 507ms
170:	learn: 0.8787469	total: 2.89s	remaining: 490ms
171:	learn: 0.8787879	total: 2.91s	remaining: 473ms
172:	learn: 0.8787879	total: 2.92s	remaining: 456ms
173:	learn: 0.8790745	total: 2.93s	remaining: 438ms
174:	learn: 0.8791155	total: 2.95s	remaining: 422ms
175:	learn: 0.8791155	total: 2.97s	remaining: 405ms
176:	learn: 0.8789926	total: 2.98s	remaining: 387ms
177:	learn: 0.8790745	total: 3s	remaining: 370ms
178:	learn: 0.8792793	total: 3.01s	remaining: 354ms
179:	learn: 0.8793612	total: 3.03s	remaining: 336ms
180:	learn: 0.87

In [43]:
model6 = CatBoostClassifier(iterations=200,learning_rate=0.1, depth=5,l2_leaf_reg=0.8,random_strength=1,
                           border_count=254,bagging_temperature = 0, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')
#params6 = {'learning_rate':[0.1,0.2,0.3], 'iterations': range(100, 255, 50)}
params6 = {'iterations':range(235,246)}
hypertune(model6,params6)

Fitting 30 folds for each of 11 candidates, totalling 330 fits
0:	learn: 0.8384111	total: 18.9ms	remaining: 4.53s
1:	learn: 0.8434889	total: 70.3ms	remaining: 8.4s
2:	learn: 0.8445946	total: 110ms	remaining: 8.75s
3:	learn: 0.8481572	total: 128ms	remaining: 7.56s
4:	learn: 0.8507371	total: 141ms	remaining: 6.64s
5:	learn: 0.8504095	total: 151ms	remaining: 5.91s
6:	learn: 0.8514333	total: 162ms	remaining: 5.4s
7:	learn: 0.8514742	total: 175ms	remaining: 5.09s
8:	learn: 0.8530303	total: 186ms	remaining: 4.81s
9:	learn: 0.8533989	total: 199ms	remaining: 4.61s
10:	learn: 0.8536036	total: 212ms	remaining: 4.44s
11:	learn: 0.8542588	total: 229ms	remaining: 4.38s
12:	learn: 0.8549550	total: 244ms	remaining: 4.28s
13:	learn: 0.8547912	total: 256ms	remaining: 4.15s
14:	learn: 0.8547912	total: 270ms	remaining: 4.07s
15:	learn: 0.8546274	total: 289ms	remaining: 4.07s
16:	learn: 0.8548731	total: 324ms	remaining: 4.27s
17:	learn: 0.8551188	total: 370ms	remaining: 4.58s
18:	learn: 0.8556511	total: 4

169:	learn: 0.8786650	total: 2.79s	remaining: 1.16s
170:	learn: 0.8787469	total: 2.8s	remaining: 1.15s
171:	learn: 0.8787879	total: 2.82s	remaining: 1.13s
172:	learn: 0.8787879	total: 2.83s	remaining: 1.11s
173:	learn: 0.8790745	total: 2.84s	remaining: 1.09s
174:	learn: 0.8791155	total: 2.86s	remaining: 1.08s
175:	learn: 0.8791155	total: 2.88s	remaining: 1.06s
176:	learn: 0.8789926	total: 2.89s	remaining: 1.04s
177:	learn: 0.8790745	total: 2.9s	remaining: 1.03s
178:	learn: 0.8792793	total: 2.92s	remaining: 1.01s
179:	learn: 0.8793612	total: 2.93s	remaining: 994ms
180:	learn: 0.8793612	total: 2.95s	remaining: 977ms
181:	learn: 0.8793612	total: 2.96s	remaining: 960ms
182:	learn: 0.8794431	total: 2.98s	remaining: 944ms
183:	learn: 0.8794840	total: 2.99s	remaining: 927ms
184:	learn: 0.8794431	total: 3.01s	remaining: 910ms
185:	learn: 0.8795659	total: 3.02s	remaining: 894ms
186:	learn: 0.8795250	total: 3.03s	remaining: 876ms
187:	learn: 0.8794840	total: 3.05s	remaining: 860ms
188:	learn: 0.

In [44]:
model = CatBoostClassifier(iterations=241,learning_rate=0.1, depth=5,l2_leaf_reg=0.8,random_strength=1,
                           border_count=254,bagging_temperature = 0, cat_features = nominal,
                            loss_function='Logloss', eval_metric='Accuracy')

In [45]:
model.fit(X_train, y_train)

0:	learn: 0.8384111	total: 14.9ms	remaining: 3.56s
1:	learn: 0.8434889	total: 28.2ms	remaining: 3.37s
2:	learn: 0.8445946	total: 42.8ms	remaining: 3.39s
3:	learn: 0.8481572	total: 54ms	remaining: 3.2s
4:	learn: 0.8507371	total: 65.4ms	remaining: 3.09s
5:	learn: 0.8504095	total: 75.4ms	remaining: 2.95s
6:	learn: 0.8514333	total: 87.3ms	remaining: 2.92s
7:	learn: 0.8514742	total: 99.8ms	remaining: 2.91s
8:	learn: 0.8530303	total: 112ms	remaining: 2.88s
9:	learn: 0.8533989	total: 123ms	remaining: 2.85s
10:	learn: 0.8536036	total: 136ms	remaining: 2.84s
11:	learn: 0.8542588	total: 149ms	remaining: 2.85s
12:	learn: 0.8549550	total: 162ms	remaining: 2.84s
13:	learn: 0.8547912	total: 174ms	remaining: 2.81s
14:	learn: 0.8547912	total: 186ms	remaining: 2.81s
15:	learn: 0.8546274	total: 199ms	remaining: 2.8s
16:	learn: 0.8548731	total: 212ms	remaining: 2.8s
17:	learn: 0.8551188	total: 229ms	remaining: 2.84s
18:	learn: 0.8556511	total: 248ms	remaining: 2.89s
19:	learn: 0.8567158	total: 262ms	rema

175:	learn: 0.8791155	total: 2.49s	remaining: 919ms
176:	learn: 0.8789926	total: 2.5s	remaining: 903ms
177:	learn: 0.8790745	total: 2.51s	remaining: 889ms
178:	learn: 0.8792793	total: 2.53s	remaining: 876ms
179:	learn: 0.8793612	total: 2.54s	remaining: 862ms
180:	learn: 0.8793612	total: 2.56s	remaining: 848ms
181:	learn: 0.8793612	total: 2.57s	remaining: 833ms
182:	learn: 0.8794431	total: 2.58s	remaining: 818ms
183:	learn: 0.8794840	total: 2.59s	remaining: 803ms
184:	learn: 0.8794431	total: 2.6s	remaining: 789ms
185:	learn: 0.8795659	total: 2.62s	remaining: 774ms
186:	learn: 0.8795250	total: 2.63s	remaining: 759ms
187:	learn: 0.8794840	total: 2.64s	remaining: 745ms
188:	learn: 0.8796069	total: 2.66s	remaining: 732ms
189:	learn: 0.8798116	total: 2.67s	remaining: 717ms
190:	learn: 0.8796478	total: 2.69s	remaining: 703ms
191:	learn: 0.8797297	total: 2.7s	remaining: 689ms
192:	learn: 0.8797297	total: 2.71s	remaining: 675ms
193:	learn: 0.8798935	total: 2.73s	remaining: 661ms
194:	learn: 0.8

In [46]:
y_predict = model.predict(X_test)

In [47]:
print('testing accuracy score:',accuracy_score(y_test,y_predict))

testing accuracy score: 0.8748157248157248


In [62]:
model

In [63]:
print(model)